In [ ]:
import sys
sys.path.append('../')
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Imputer
from util import *
from onehot import LabelBinarizerEx
from pipeline import FeaturePipeline, DataFramePipeline
from binning import Binner
from ensemble import EnsembleStackClassifierEx, StackRegressor
from addcols import AddColumns
from impute import GroupImputer
from ordinal import Ordinar

%matplotlib inline
from matplotlib import pyplot

## explore data

In [ ]:
import pandas as pd
house_train=pd.read_csv('train.csv')
target_col='SalePrice'
id_col='Id'
target=house_train[target_col]#/house_train['LotArea']
houses=house_train.copy()

In [ ]:
num_summary=houses.describe()
num_summary

In [ ]:
cat_summary=houses.describe(include=['O'])
cat_summary

add unit price

In [ ]:
houses['UnitPrice']=houses[target_col]/houses['LotArea']

explore features

In [ ]:
corrs=houses.corr()
corrs[['SalePrice','UnitPrice']].sort_values(['UnitPrice'],ascending=False)

In [ ]:
df=houses.groupby('LotShape').count()[[id_col]].sort_values(id_col)
df.plot(kind='bar')

In [ ]:
df=houses.groupby('MSSubClass').mean()[['UnitPrice']].sort_values('UnitPrice')
df.plot(kind='bar')

LotShape is ordinal

'Reg' is max

In [ ]:
# corrs['UnitPrice']['LotFrontage']

LotFrontage is slight neg corr with up

it should be discretized

In [ ]:
# corrs['UnitPrice']['LotArea']

same with LotFrontage

In [ ]:
# df=house_train.groupby('Utilities').mean()[['UnitPrice']]
# df.sort_values('UnitPrice').plot(kind='bar')

make Utilities ordinal

In [ ]:
# df=house_train.groupby('LandSlope').mean()[['UnitPrice']]
# df.sort_values('UnitPrice').plot(kind='bar')

make LandSlope ordinal

merge condition1&2 into 1 col

In [ ]:
# df=house_train.groupby('HouseStyle').mean()[['UnitPrice']].sort_values('UnitPrice')
# df.plot(kind='bar')

make HouseStyle ordinal , SLvl is max

In [ ]:
# corrs['UnitPrice']['YearRemodAdd']

discretize YearBuilt, band is 9

In [ ]:
# house_train['YearTotal']=house_train['YearRemodAdd']+house_train['YearBuilt']
# house_train['YearTotalBand']=pd.qcut(house_train['YearTotal'],6,labels=False,duplicates='drop')
# corrs=house_train.corr()
# corrs['UnitPrice'][['YearBuilt','YearBand','YearTotal','YearRemodAdd','YearTotalBand']]

add YearBuilt & YearRemodAdd to make YearTotal, discretize it

make ExterQual,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,FireplaceQu,GarageFinish, GarageQual, GarageCond, PoolQC, Fence, ordinal

In [ ]:
# df=house_train.groupby('RoofStyle').mean()[['UnitPrice']].sort_values('UnitPrice')
# df.plot(kind='bar')

In [ ]:
import pandas as pd
import numpy as np
import math
from sys import float_info

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error,mean_squared_error

class Binner(BaseEstimator,TransformerMixin):
    '''
    binner for continuous feature
    '''
    def __init__(self,strategy=None):
        self.strategy=strategy
        
    def fit(self, X, y=None):
        if self.strategy is None and y is not None:
            min_err=float_info.max
            best_q=0
            for q in range(2,10):
                s=pd.qcut(X,q,labels=False,duplicates='drop')
                err=mean_absolute_error(s,y)
                print(err)
                if err < min_err:
                    best_q=q
                    min_err=err
            print('best q: %d'% q)
        return self
    
b=Binner()
b.fit(houses['YearBuilt'],houses[target_col])

In [ ]:
houses.describe()[['MasVnrArea']]

In [ ]:
houses['MasVnrArea'].hist(bins=100)

In [ ]:
houses['LotArea_sqrt']=np.sqrt(houses['LotArea'])
houses['LotArea_sqrt'].hist(bins=100)

## feature enginering

In [ ]:
drop_cols=[id_col,target_col]

In [ ]:
num_cols=list(num_summary.columns)
num_cols=list(filter(lambda c: c not in drop_cols, num_cols))
# num_cols.remove('MSSubClass')

In [ ]:
num_cont_cols=list(filter(lambda c: len(house_train[c].unique())>20 , num_cols))
num_disc_cols=set(num_cols)-set(num_cont_cols)
print(len(num_disc_cols),len(num_cont_cols))

In [ ]:
cat_cols=list(cat_summary.columns)
# cat_cols.append('MSSubClass')

In [ ]:
cat_ordinals={
    'LotShape':['IR3','IR2','IR1','Reg'],
    'Utilities':['ELO','NoSeWa','NoSewr','AllPub'],
    'LandSlope':['Sev','Mod','Gtl',],
    'HouseStyle':['1Story','1.5Fin','1.5Unf','2Story','2.5Fin','2.5Unf','SFoyer','SLvl'],
    'ExterQual':['po','fa','ta','gd','ex'],
    'BsmtQual':['po','fa','ta','gd','ex'],
    'BsmtCond':['po','fa','ta','gd','ex'],
    'BsmtExposure':['no','mn','av','gd'],
    'BsmtFinType1':['unf','lwq','rec','blq','alq','glq'],
    'BsmtFinType2':['unf','lwq','rec','blq','alq','glq'],
    'HeatingQC':['po','fa','ta','gd','ex'],
    'KitchenQual':['po','fa','ta','gd','ex'],
    'Functional':['sal','sev','maj2','maj1','mod','min2','min1','typ'],
    'FireplaceQu':['po','fa','ta','gd','ex'],
    'GarageFinish':['unf','rfn','fin'], 
    'GarageQual':['po','fa','ta','gd','ex'],
    'GarageCond':['po','fa','ta','gd','ex'], 
    'PoolQC':['fa','ta','gd','ex'], 
    'Fence':['mnww','gdwo','mnprv','gdprv']
}
cat_ord_cols=cat_ordinals.keys()

In [ ]:
cat_nom_cols=set(cat_cols)-cat_ord_cols
print(len(cat_ord_cols),len(cat_nom_cols))

In [ ]:
num_pipelines=[FeaturePipeline(c,c,Pipeline([('impute',Imputer(strategy='median'))])) 
               for c in num_cols]
cat_nom_pipelines=[FeaturePipeline(c,'',Pipeline([('onehot',LabelBinarizerEx([c]))])) for c in cat_nom_cols]
cat_ord_pipelines=[FeaturePipeline(c,c,Pipeline([('ordinal',Ordinar(cat_ordinals[c]))])) 
                       for c in cat_ord_cols]
full_pipeline=DataFramePipeline(num_pipelines+cat_nom_pipelines+cat_ord_pipelines)
prepared_house_train=full_pipeline.fit_transform(house_train)
prepared_house_train.drop(drop_cols+list(cat_nom_cols),axis=1,inplace=True)
prepared_house_train.head()

## train

In [ ]:
from sklearn.linear_model import SGDClassifier,LinearRegression,ElasticNet
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor

base_regs=[
    ElasticNet(random_state=0),
#     LinearRegression(n_jobs=-1),
#     SVC(probability=True),
    RandomForestRegressor(n_jobs=-1,random_state=0),
    ExtraTreesRegressor(n_jobs=-1,random_state=0),
    GradientBoostingRegressor(random_state=0),
    XGBRegressor(seed=0),
]

In [ ]:
from sklearn.model_selection import cross_val_score
from math import sqrt,log10

%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *

def trainModels(train_data, target):
    scores=[log10(sqrt(-cross_val_score(reg,train_data,target,scoring='neg_mean_squared_error',cv=5,n_jobs=-1,verbose=1).mean())) 
            for reg in base_regs]

    labels=[c.__class__.__name__[:3] for c in base_regs]
    X=np.arange(len(base_regs))
    bar(X,scores,tick_label=labels,color='rgb')
#     ylim(0.5,1.0)
    show()
    print(sorted(zip(labels,scores),key=lambda x:x[1],reverse=True))

In [ ]:
trainModels(prepared_house_train,target)

In [ ]:
best_reg=GradientBoostingRegressor()
best_reg.fit(prepared_house_train,target)


## stacking

In [ ]:
from sklearn.linear_model import ElasticNet
stack=StackRegressor(base_regs,LinearRegression(),0.99)
log10(sqrt(-cross_val_score(stack,prepared_house_train,target,scoring='neg_mean_squared_error',cv=5,n_jobs=-1,verbose=1).mean()))

In [ ]:
best_reg=stack
best_reg.fit(prepared_house_train,target)

## tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid_set=[
#                 {'C':[0.01,0.1,0.5,1.]},
#                 {'C':[1.,10.,],'kernel':['rbf','poly'],'gamma':[0.01,0.1,1.],'coef0':[1.,10.,]},
#                 {'n_estimators':[50,100,200,300],'max_depth':[5,10,15]},
                {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,300,500,1000],'max_depth':[3,5,8]},
#                 {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,200,300]},
#                 {'n_estimators':[50,100,200,300],'max_depth':[5,10,15]},
#                 {'learning_rate':[0.01,0.1,1.0],'n_estimators':[100,200,300],'max_depth':[5,10,15],'gamma':[0.01,0.1,0.5]},
               ]

def tuneModels(train_data,target):
    results=[]
    for i in range(len(base_regs)):
        gs=GridSearchCV(estimator=base_regs[i],param_grid=param_grid_set[i],scoring='neg_mean_squared_error',n_jobs=-1,verbose=1,cv=5)
        gs.fit(train_data,target)
        results.append((gs.best_estimator_,gs.best_score_))
    print(sorted(results,key=lambda x:x[1],reverse=True))
    return results

In [ ]:
# results=tuneModels(prepared_house_train,target)
# best_reg=results[0][0]

## test

In [ ]:
house_test=pd.read_csv('test.csv')
test_drop_cols=list(filter(lambda c: c != target_col, drop_cols))

In [ ]:
test_id=house_test[id_col]
prepared_house_test=full_pipeline.transform(house_test)
prepared_house_test.drop(test_drop_cols+list(cat_nom_cols),axis=1,inplace=True)


In [ ]:
prices=best_reg.predict(prepared_house_test)
df=pd.DataFrame({id_col:test_id,target_col:prices.astype(int)})
df.to_csv('result.csv',index=False)